In [85]:
import pandas as pd
from sklearn.linear_model import LinearRegression

In [ ]:
# makine ogrenmesi FORMUL  y = m1x1 + m2x2 + mnxn + b

In [89]:
# https://www.kaggle.com/datasets/jacklacey/audi-a1-listings/data adresinden Audi_A1_listings.csv datadini indirdik
#pd.read_cvs ile dizindeki audi.csv dosyamizi okuyup df degiskenine taniimliyoruz
df = pd.read_csv('audi.csv')
df.head(3)

,index,Year,Type,Mileage(miles),Engine,PS,Transmission,Fuel,Number_of_Owners,Price(£),href,PPY,MileageRank,PriceRank,PPYRank,Score
0,0,2018.0,Hatchback,44000.0,1.6L,114.398422,Manual,Diesel,1,14995.0,https://www.autotrader.co.uk/car-details/20221...,2499.166667,215,163,340,718
1,4,2016.0,Hatchback,42596.0,1.0L,93.688363,Manual,Petrol,3,10755.0,https://www.autotrader.co.uk/car-details/20221...,2688.750000,222,330,276,828
2,7,2015.0,Hatchback,42700.0,1.4L,123.274162,Manual,Petrol,2,10799.0,https://www.autotrader.co.uk/car-details/20221...,3599.666667,221,327,94,642


In [90]:
# makine ogrenmesi icin ihtiyacimiz olmayan sutunlari temizleyelim
#bu datada fiyat tahmini icin araba linklerine ranklara ihtiyacim yok
df = df.drop(columns= ['index','href','MileageRank','PriceRank','PPYRank','Score'])
df.head(3)

,Year,Type,Mileage(miles),Engine,PS,Transmission,Fuel,Number_of_Owners,Price(£),PPY
0,2018.0,Hatchback,44000.0,1.6L,114.398422,Manual,Diesel,1,14995.0,2499.166667
1,2016.0,Hatchback,42596.0,1.0L,93.688363,Manual,Petrol,3,10755.0,2688.750000
2,2015.0,Hatchback,42700.0,1.4L,123.274162,Manual,Petrol,2,10799.0,3599.666667


In [91]:
# Type , Transmission, Fuel gibi veriler KATEGORISEL rakamsal veriler ise NUMERIK verilerdir.

# y = mx + b  Y =tahmin edilecek veri x = etki katsayisi m = etki eden faktor (dizel, motor, yil vb. her biri ayri m1x1 m2x2 mnxn) b = sabit
# Data mizi Numerik veriye donusturmemiz gerekiyor.

# Datamizin neleri numerige cevirmemiz gerektigini ogrenmek icin pd.info()
df.info()
#Fload ve int harici butun verilerimiz Kategorisel veri imis

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471 entries, 0 to 470
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Year              471 non-null    float64
 1   Type              471 non-null    object 
 2   Mileage(miles)    471 non-null    float64
 3   Engine            471 non-null    object 
 4   PS                471 non-null    float64
 5   Transmission      471 non-null    object 
 6   Fuel              471 non-null    object 
 7   Number_of_Owners  471 non-null    int64  
 8   Price(£)          471 non-null    float64
 9   PPY               471 non-null    float64
dtypes: float64(5), int64(1), object(4)
memory usage: 36.9+ KB


In [92]:
# oncelikle sutun adlarini degistirmekle baslayalim
# oncelikle Engin sutununu NUmerik veriye donustirmekle baslayalim
df.columns=['yil','kasa','mil','motor','ps','vites','yakit','sahip','fiyat','ppy']
df.head(3)

,yil,kasa,mil,motor,ps,vites,yakit,sahip,fiyat,ppy
0,2018.0,Hatchback,44000.0,1.6L,114.398422,Manual,Diesel,1,14995.0,2499.166667
1,2016.0,Hatchback,42596.0,1.0L,93.688363,Manual,Petrol,3,10755.0,2688.750000
2,2015.0,Hatchback,42700.0,1.4L,123.274162,Manual,Petrol,2,10799.0,3599.666667


In [93]:
# motor sutundaki L ibarelerini str.replace komutuyla degistiriyorum
df['motor'] = df['motor'].str.replace('L','')
df.head(3)
#numerige benzedi fakat sutun henuz object olarak tanimli. int yada fload a cevirmek gerekiyor

,yil,kasa,mil,motor,ps,vites,yakit,sahip,fiyat,ppy
0,2018.0,Hatchback,44000.0,1.6,114.398422,Manual,Diesel,1,14995.0,2499.166667
1,2016.0,Hatchback,42596.0,1.0,93.688363,Manual,Petrol,3,10755.0,2688.750000
2,2015.0,Hatchback,42700.0,1.4,123.274162,Manual,Petrol,2,10799.0,3599.666667


In [94]:
#numerige donuzmeye uygunlastirdigimiz motor sutunumuzu  to_numeric comutuyla numerige donusturuyoruz
df['motor'] = pd.to_numeric( df ['motor'] )
df['motor'].head(3)
# object olan motor sutunumuz artik fload

0    1.6
1    1.0
2    1.4
Name: motor, dtype: float64

In [95]:
# simdi direk kategorisel verileri numerige cevirecegiz get_dummies comutuyla
df= pd.get_dummies(df,columns=['kasa','vites','yakit'],drop_first=True) # drop_first=True iki veri yi teke cevir- yani oto vites ile man vites ayri olmaasin. oto vites 0 ise man vites demektir
df.head(5)
# Artik verimiz model olusturmak icin hazir. butun sutunlari NUMERIC

,yil,mil,motor,ps,sahip,fiyat,ppy,vites_Manual,yakit_Petrol
0,2018.0,44000.0,1.6,114.398422,1,14995.0,2499.166667,True,False
1,2016.0,42596.0,1.0,93.688363,3,10755.0,2688.750000,True,True
2,2015.0,42700.0,1.4,123.274162,2,10799.0,3599.666667,True,True
3,2014.0,86000.0,1.6,103.550296,3,7490.0,3745.000000,True,False
4,2014.0,104310.0,1.6,103.550296,3,7400.0,3700.000000,True,False


In [99]:
# datamiz hazir. makine ogrenmesine gecebiliriz
# oncelikle x ve y mizi tanimliyoruz
# y = mx+b y= tahmin etmek istedigimiz veri = fiyat
y = df[['fiyat']]
# x diger butun etkenler x
x = df.drop('fiyat', axis = 1 ) # fiyat haricinde hepsi x olsun anlamina gelir yada x = df[['yil','motor',vs, vs]] yazabilir
df.head()

,yil,mil,motor,ps,sahip,fiyat,ppy,vites_Manual,yakit_Petrol
0,2018.0,44000.0,1.6,114.398422,1,14995.0,2499.166667,True,False
1,2016.0,42596.0,1.0,93.688363,3,10755.0,2688.750000,True,True
2,2015.0,42700.0,1.4,123.274162,2,10799.0,3599.666667,True,True
3,2014.0,86000.0,1.6,103.550296,3,7490.0,3745.000000,True,False
4,2014.0,104310.0,1.6,103.550296,3,7400.0,3700.000000,True,False


In [100]:
# modelimizi olusturuyoruz
lm = LinearRegression()
model = lm.fit(x,y) #modeli fit ediyoruz ogretiyoruz

In [101]:
# lm miz ogrendi, simdi bir tahmin yaptiralim predict comutuyla
model.predict([[2017,50000,1.6,103.550296,1,3800,0,1]])#yil	mil	motor	ps	sahip	FIYAT=X=YAZMA	ppy	vites_Manual	yakit_Petrol
# bu veri ye gore tahminimiz 14932 euro

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([[14931.94147347]])

In [102]:
# tahminler ne derece dogru
model.score(x,y)
# % 96 civarinda dogru tahmin ediyormus

0.9595295772839731

SyntaxError: invalid syntax (<ipython-input-105-7807c62fd889>, line 1)